## "Hello World" in PyTorch

In [ ]:
# Imports torch
import torch
from torch import nn

# Defines our first network
class FirstNetwork(nn.Module):
    def __init__(self, multiplier):
        super().__init__()
        self.multiplier = nn.Parameter(torch.tensor(float(multiplier)))
    
    def forward(self, x):
        return self.multiplier * x

# Initializes FirstNetwork
network = FirstNetwork(2)
print(network)

# Runs the network on an input and gets the result
network(4).item()



In [ ]:
# Defines a logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_features, class_num):
        super().__init__()
        self.weights = nn.Linear(input_features, class_num)
    
    def forward(self, x):
        return self.weights(x)
    
    def get_probabilities(self, x):
        p = nn.functional.softmax(self(x), dim=-1)
        return p.detach().numpy()

# Initializes LogisticRegression with 3 features and 2 classes
lr_network = LogisticRegression(3, 2)
print(lr_network)

# Defines some fake data for testing
# 2 examples with 3 features each
test_data = torch.tensor([[1.0, 2.0, 3.0],
                         [3.0, 2.0, 1.0]])
print(test_data)

# Gets the probabilities for each example for each class
lr_network.get_probabilities(test_data)



In [ ]:
# Defines the loss function
loss_func = nn.CrossEntropyLoss()

# Defines some fake labels for the fake data
test_labels = torch.tensor([0, 1])

# Calls the network on the test data (using forward)
output = lr_network(test_data)

# Checks the shape of the output and labels
# (Just for debugging)
print(output.shape)
print(test_labels.shape)

# Computes the average loss over examples
loss_func(output, test_labels).item()



In [ ]:
# Defines some random train and test data, where the positive class has
# higher average value for the first feature
# 
# Don't worry too much about this code
import numpy as np
from numpy.random import default_rng
X_train = default_rng(0).standard_normal([100, 3])
X_test = default_rng(1).standard_normal([100, 3])
for i in range(50):
    X_train[i, 0] += 1.5
    X_test[i, 0] += 1.5
Y_train = np.concatenate([np.ones(50), np.zeros(50)])
Y_test = np.concatenate([np.ones(50), np.zeros(50)])

print(np.mean(X_train[:50],axis=0))
print(np.mean(X_train[50:],axis=0))
print()
print(Y_train)



In [ ]:
# Wraps the data in PyTorch datasets
from torch.utils.data import TensorDataset
dataset_train = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(Y_train, dtype=torch.long))

dataset_test = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(Y_test, dtype=torch.long))

print(len(dataset_train))
print(len(dataset_test))

# Creates data loaders to batch up the data
from torch.utils.data import DataLoader
dataloader_train = DataLoader(dataset_train, batch_size=10, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=10, shuffle=False)

for X, Y in dataloader_train:
    print(X)
    print(Y)
    print()



In [ ]:
# Creates an optimizer
from torch.optim import SGD
lr_optimizer = SGD(lr_network.parameters(), lr=0.01)
print(lr_optimizer)

# Iteratively minimizes training loss
epochs = 500
lr_network.train()

for _ in range(epochs):
    for X, Y in dataloader_train:
        # Zeros out gradient information
        lr_optimizer.zero_grad()
        
        # Computes the training loss on the batch
        outputs = lr_network(X)
        loss = loss_func(outputs, Y)
        
        # Computes gradient
        loss.backward()
        
        # Takes optimization step
        lr_optimizer.step()


In [ ]:
# Computes test loss
lr_test_loss = 0
lr_network.eval()

with torch.no_grad():
    for X, Y in dataloader_test:
        # Computes the training loss on the batch
            outputs = lr_network(X)
            loss = loss_func(outputs, Y)

            lr_test_loss += loss.item() * X.shape[0]

print(lr_test_loss)


In [ ]:
# Defines a multilayer perceptron model
class MLP(nn.Module):
    def __init__(self, input_features, hidden_size, class_num):
        super().__init__()
        self.layer1 = nn.Linear(input_features, hidden_size)
        self.activation1 = nn.Sigmoid()
        self.layer2 = nn.Linear(hidden_size, class_num)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.activation1(out)
        out = self.layer2(out)
        return out
    
    def get_probabilities(self, x):
        p = nn.functional.softmax(self(x), dim=-1)
        return p.detach().numpy()

# Initializes MLP with 3 features, 10 hidden units, and 2 classes
mlp_network = MLP(3, 10, 2)
print(mlp_network)



In [ ]:
# Trains the MLP
mlp_optimizer = SGD(mlp_network.parameters(), lr=0.01)
mlp_network.train()

for _ in range(epochs):
    for X, Y in dataloader_train:
        # Zeros out gradient information
        mlp_optimizer.zero_grad()
        
        # Computes the training loss on the batch
        outputs = mlp_network(X)
        loss = loss_func(outputs, Y)
        
        # Computes gradient
        loss.backward()
        
        # Takes optimization step
        mlp_optimizer.step()



In [ ]:
# Computes test loss for MLP
mlp_test_loss = 0
mlp_network.eval()

with torch.no_grad():
    for X, Y in dataloader_test:
        # Computes the loss on the batch
        outputs = mlp_network(X)
        loss = loss_func(outputs, Y)
        
        mlp_test_loss += loss.item() * X.shape[0]

print(mlp_test_loss)

if mlp_test_loss < lr_test_loss:
    print("MLP has lower test loss!")
else:
    print("LogisticRegression has lower test loss!")



## Let's go back to Lecture 18, slide 12!

In [ ]:
# Defines an input tensor of 10 x 10 "images" with 2 examples 3 color channels
color_channels = 3
imgs = default_rng(2).standard_normal([2, color_channels, 10, 10])
imgs = torch.tensor(imgs, dtype=torch.float32)

# Defines some 2d convolutional layers and compares the shapes of the outputs
out_channels = 1
kernel_size = 1
stride = 1

conv = nn.Conv2d(
    color_channels, out_channels, kernel_size=kernel_size, stride=stride
)
print(conv)
print(conv(imgs).shape)

# Flattens the output so that we can treat features as a vector
flatten = nn.Flatten()
print(flatten(conv(imgs)).shape)


